In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10
data = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = data.load_data()
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

# Normalize
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

In [7]:
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(x_train)

In [8]:
# Custom CNN Model
def build_strong_cnn():
    model = models.Sequential([
        layers.Conv2D(64, 3, padding='same', activation='relu', input_shape=(32, 32, 3)),
        layers.BatchNormalization(),
        layers.Conv2D(64, 3, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),
        layers.Dropout(0.3),

        layers.Conv2D(128, 3, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(128, 3, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),
        layers.Dropout(0.4),

        layers.Conv2D(256, 3, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),
        layers.Dropout(0.4),

        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [10]:
# Train Custom CNN
cnn_model = build_strong_cnn()
train_flow = datagen.flow(x_train, y_train_cat, batch_size=64)
steps = len(x_train) // 64
cnn_model.fit(train_flow,
              steps_per_epoch=steps,
              epochs=50,
              validation_data=(x_test, y_test_cat),
              callbacks=[EarlyStopping(patience=7, restore_best_weights=True)])

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


781/781 ━━━━━━━━━━━━━━━━━━━━ 49s 49ms/step - accuracy: 0.2651 - loss: 2.1532 - val_accuracy: 0.4724 - val_loss: 1.4704
Epoch 2/50
  1/781 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.3906 - loss: 1.5699

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3906 - loss: 1.5699 - val_accuracy: 0.4709 - val_loss: 1.4576
Epoch 3/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 31s 40ms/step - accuracy: 0.4310 - loss: 1.5649 - val_accuracy: 0.5443 - val_loss: 1.3267
Epoch 4/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4688 - loss: 1.4598 - val_accuracy: 0.5416 - val_loss: 1.3229
Epoch 5/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 39s 39ms/step - accuracy: 0.5150 - loss: 1.3499 - val_accuracy: 0.4093 - val_loss: 1.9003
Epoch 6/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4844 - loss: 1.4037 - val_accuracy: 0.4013 - val_loss: 1.9077
Epoch 7/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 41s 41ms/step - accuracy: 0.5711 - loss: 1.2109 - val_accuracy: 0.6057 - val_loss: 1.1407
Epoch 8/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5625 - loss: 1.1708 - val_accuracy: 0.6084 - val_loss: 1.1293
Epoch 9/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 31s 40ms/step - accuracy: 0.6148 - loss: 1.1177 - val_accuracy: 0.684

In [11]:
cnn_acc = cnn_model.evaluate(x_test, y_test_cat, verbose=0)[1]
print(f"Custom CNN Test Accuracy: {cnn_acc * 100:.2f}%")

Custom CNN Test Accuracy: 84.51%


In [12]:
# Transfer Learning with VGG16

def build_finetuned_vgg():
    base_model = VGG16(include_top=False, weights='imagenet', input_shape=(32, 32, 3))
    for layer in base_model.layers[-4:]:
        layer.trainable = True

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
                  loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [13]:
# Train VGG16 model
vgg_model = build_finetuned_vgg()
vgg_model.fit(train_flow,
              steps_per_epoch=steps,
              epochs=50,
              validation_data=(x_test, y_test_cat),
              callbacks=[EarlyStopping(patience=7, restore_best_weights=True)])

Epoch 1/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 60s 64ms/step - accuracy: 0.5014 - loss: 1.4271 - val_accuracy: 0.7579 - val_loss: 0.7306
Epoch 2/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7031 - loss: 0.8554 - val_accuracy: 0.7620 - val_loss: 0.7219
Epoch 3/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 44s 56ms/step - accuracy: 0.7659 - loss: 0.7225 - val_accuracy: 0.8104 - val_loss: 0.5688
Epoch 4/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7031 - loss: 0.8890 - val_accuracy: 0.8148 - val_loss: 0.5631
Epoch 5/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 80s 56ms/step - accuracy: 0.8054 - loss: 0.5956 - val_accuracy: 0.8286 - val_loss: 0.5085
Epoch 6/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8750 - loss: 0.4579 - val_accuracy: 0.8280 - val_loss: 0.5111
Epoch 7/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 79s 56ms/step - accuracy: 0.8327 - loss: 0.5056 - val_accuracy: 0.8375 - val_loss: 0.5005
Epoch 8/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7656 - loss: 0.6160 - val_accu

In [14]:
vgg_acc = vgg_model.evaluate(x_test, y_test_cat, verbose=0)[1]
print(f"VGG16 Transfer Learning Test Accuracy: {vgg_acc * 100:.2f}%")

VGG16 Transfer Learning Test Accuracy: 87.73%


|  **Aspect**            |  **Custom CNN**                               |  **VGG16 (Transfer Learning)**                      |
| ------------------------ | ----------------------------------------------- | ----------------------------------------------------- |
| **Model Type**           | CNN built from scratch                          | Pre-trained model fine-tuned on CIFAR-10              |
| **Architecture**         | \~9–12 layers (Conv, Pooling, Dense)            | 16 layers (13 Conv + 3 Dense), pretrained on ImageNet |
| **Pretraining Used?**    |  No                                            |  Yes (ImageNet)                                      |
| **Trainable Parameters** |  Fewer (Lightweight model)                    |  More (Heavy model with millions of parameters)     |
| **Training Time**        | Fast                                          |  Slower due to model size                            |
| **Feature Learning**     | Learns everything from scratch                  | Uses high-quality pre-trained visual features         |
| **Generalization Power** | Moderate (depends on regularization techniques) | High (benefits from pretrained filters)               |
| **Accuracy Achieved**    | **84.51%**                                      | **87.73%**                                            |
| **Overfitting Risk**     | Medium                                          | Lower (due to good base features)                     |


VGG16 outperformed the custom CNN because it uses pretrained features from ImageNet, allowing it to recognize patterns more effectively without learning from scratch. Its deeper architecture captures more complex features, and fine-tuning only the top layers helps it adapt well to CIFAR-10 with less overfitting. In contrast, the custom CNN, being shallower and untrained, had to learn all features from scratch, limiting its performance.
